Consultar o valor do Dolar por dia com API

In [1]:
import pandas as pd
import requests
from datetime import date

In [6]:

ini = "04-01-2025"
fim = date.today().strftime("%m-%d-%Y")
url = (
  "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
  f"CotacaoDolarPeriodo(dataInicial=@ini,dataFinalCotacao=@fim)?"
  f"@ini='{ini}'&@fim='{fim}'&$select=cotacaoCompra,dataHoraCotacao&$top=10000&$format=json"
)

data = requests.get(url, timeout=30).json()["value"]
df = pd.DataFrame(data)
df["data"] = pd.to_datetime(df["dataHoraCotacao"]).dt.date

# última cotação do dia (fechamento)
df_daily = df.sort_values("dataHoraCotacao").groupby("data").tail(1)
df_daily = df_daily[["data","cotacaoCompra"]].reset_index(drop=True)

# formata data para dd/mm/yyyy
df_daily["data"] = pd.to_datetime(df_daily["data"]).dt.strftime("%d/%m/%Y")
df_daily

,data,cotacaoCompra
0,01/04/2025,5.7045
1,02/04/2025,5.6918
2,03/04/2025,5.6062
3,04/04/2025,5.7771
4,07/04/2025,5.8871
...,...,...
140,20/10/2025,5.3765
141,21/10/2025,5.3842
142,22/10/2025,5.3892
143,23/10/2025,5.3834


In [8]:
import duckdb

DB_PATH = "dados_dolar.duckdb"
con = duckdb.connect(DB_PATH)

# 1) cria tabela (se não existir)
con.execute("""
CREATE TABLE IF NOT EXISTS dolar_diario (
  data DATE,
  cotacao_compra DOUBLE
);
""")

# 2) registra o DataFrame atual como uma view temporária
con.register("df_stage", df_daily)

# 3) upsert por data (converte 'dd/mm/yyyy' -> DATE)
con.execute("""
MERGE INTO dolar_diario AS t
USING (
  SELECT
    STRPTIME(data, '%d/%m/%Y')::DATE AS data,
    CAST(cotacaoCompra AS DOUBLE)     AS cotacao_compra
  FROM df_stage
) AS s
ON t.data = s.data
WHEN MATCHED THEN UPDATE SET
  cotacao_compra = s.cotacao_compra
WHEN NOT MATCHED THEN INSERT (data, cotacao_compra)
VALUES (s.data, s.cotacao_compra);
""")

# 4) checagem rápida
print(con.execute("""
SELECT *
FROM dolar_diario
ORDER BY data DESC
LIMIT 10;
""").fetchdf())

con.execute("""
COPY dolar_diario TO './exports/dolar_diario.csv' (HEADER, DELIMITER ',');
""")

con.close()


        data  cotacao_compra
0 2025-10-24          5.3791
1 2025-10-23          5.3834
2 2025-10-22          5.3892
3 2025-10-21          5.3842
4 2025-10-20          5.3765
5 2025-10-17          5.4384
6 2025-10-16          5.4348
7 2025-10-15          5.4458
8 2025-10-14          5.4976
9 2025-10-13          5.4623
